In [2]:
import tensorflow as tf
import numpy as np
import tempfile

In [3]:
batchSize = 5
maxNumSteps = 8
embeddingSize = 3
nNeurons = 4
vocabulary_size = 10
initEmbeddings = np.random.randn(vocabulary_size, embeddingSize)
bias_trainable = False

In [4]:
tf.reset_default_graph()
inputTokens = tf.placeholder(tf.int32, [batchSize, maxNumSteps])
inputLens = tf.placeholder(tf.int32, [batchSize])
targets = tf.placeholder(tf.float64, [batchSize])
rnnCell = tf.nn.rnn_cell.BasicRNNCell(nNeurons)
initState = rnnCell.zero_state(batchSize, tf.float64)

embedding = tf.Variable(initEmbeddings, name='inputEmbeddings', trainable=False, dtype=tf.float64)
inputData = tf.nn.embedding_lookup(embedding, inputTokens)

raw_outputs, last_states = tf.nn.dynamic_rnn(
    cell=rnnCell,
    dtype=tf.float64,
    sequence_length=inputLens,
    inputs=inputData)
    
flattened_outputs = tf.reshape(raw_outputs, [-1, nNeurons])
index = tf.range(0, batchSize) * maxNumSteps + inputLens - 1
outputs = tf.gather(flattened_outputs, index)
outputW = tf.get_variable("outputW", [nNeurons, 1], dtype=tf.float64)
outputB = tf.get_variable("outputB", [1], dtype=tf.float64)
prediction = tf.add(tf.matmul(outputs, outputW), outputB)
loss = tf.reduce_sum(tf.pow(prediction-targets, 2)/batchSize)

lr = tf.Variable(0.1, trainable=False)
tvars = tf.trainable_variables()
if not bias_trainable:
    tvars = np.take(tvars, [0,2,3]).tolist() # exclude RNN bias from training
optimizer = tf.train.GradientDescentOptimizer(lr)
learningStep = optimizer.minimize(loss, var_list=tvars)
initAll = tf.initialize_all_variables()

In [5]:
# data = np.random.randn(batchSize, maxNumSteps, embeddingSize)
data = np.random.randint(0, high=vocabulary_size-1, size=(batchSize, maxNumSteps))
lens = [maxNumSteps for i in xrange(batchSize)]
t = np.random.randn(batchSize)
with tf.Session() as sess:
    sess.run(initAll)
#     e = sess.run(embedding)
#     print e
#     print data
#     s = sess.run(inputData, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print s
#     l = sess.run(loss, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print l
    for v in tf.trainable_variables():
        print v.name
        print sess.run(v)
#     w = sess.run(outputW)
#     b = sess.run(outputB)
#     print w
#     print b
#     r = sess.run(raw_outputs, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print len(r), len(r[0])
#     print r[:,-1]
#     i = sess.run(index, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print i
#     o = sess.run(outputs, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print o
#     m = sess.run(tf.matmul(outputs, outputW), feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print m
#     p = sess.run(prediction, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print p
    sess.run(learningStep, feed_dict={inputTokens:data, inputLens:lens, targets:t})
    l = sess.run(loss, feed_dict={inputTokens:data, inputLens:lens, targets:t})
    print l
    for v in tf.trainable_variables():
        print v.name
        print sess.run(v)

RNN/BasicRNNCell/Linear/Matrix:0
[[ 0.42941295 -0.07258421 -0.54979762 -0.11436226]
 [ 0.61322717  0.03764971 -0.24182697 -0.36987435]
 [-0.58862103 -0.63002914  0.09251724 -0.54323472]
 [-0.57283322  0.64178415  0.62736614  0.1091234 ]
 [-0.1387797   0.14621779  0.26264562  0.11519057]
 [ 0.43270537 -0.27682909  0.11104942  0.60778161]
 [ 0.30884604 -0.5391706  -0.00791769  0.35584711]]
RNN/BasicRNNCell/Linear/Bias:0
[ 0.  0.  0.  0.]
outputW:0
[[ 0.15560042]
 [ 0.55991761]
 [ 0.25549454]
 [ 0.28202788]]
outputB:0
[-0.83189643]
4.7232749571
RNN/BasicRNNCell/Linear/Matrix:0
[[ 0.39361403 -0.3548663  -0.64596807 -0.24474875]
 [ 0.61318321  0.15649813 -0.20737876 -0.30127714]
 [-0.57707762 -0.58114884  0.0933289  -0.53402564]
 [-0.58224216  0.69261914  0.63830254  0.15717213]
 [-0.14172636  0.09487458  0.24822881  0.07911065]
 [ 0.49856179 -0.26616252  0.13136916  0.57045825]
 [ 0.32198598 -0.48820845  0.00958399  0.39029228]]
RNN/BasicRNNCell/Linear/Bias:0
[ 0.  0.  0.  0.]
outputW:0
[[

In [33]:
np.dot(r[:,-1], w)

array([[ 0.66335526],
       [ 0.70219608],
       [-0.58211488],
       [ 0.07739599],
       [ 0.10125138]])

TypeError: list indices must be integers, not list